In [284]:
import pandas as pd

from collections import namedtuple,defaultdict
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm_notebook, tqdm
import pymorphy2
import bisect
import math

In [290]:
data = pd.read_csv('fake news data.csv', sep='\t', index_col=False )
data = data.drop(['Unnamed: 0'], axis = 1)
indexes = data.index
data.head(3)


,Header,Content,Time,Link,Label,Topic,keywords
0,"В одном из ресторанов Нью-Йорка появится ""пяти...",Нью-йоркский ресторан Lucy's Cantina Royale бу...,6/10/2017,https://tass.ru/obschestvo/4625856,0,Политика,"пауть,острый,президент,владимир,честь,ресторан..."
1,Экс-глава МИ-6 рассказал о провальном плане за...,Бывший глава британской разведки МИ-6 Джон Ска...,15/08/2017,http://ren.tv/novosti/2017-08-15/eks-glava-mi-...,0,Политика,"россия,стратегический план,развал,план,стратег..."
2,Умерла нобелевский лауреат по литературе Светл...,Нобелевский лауреат по литературе Светлана Але...,18/05/2017,https://lenta.ru/news/2017/05/18/alex_nob/,0,Знаменитости,"еврей,жизнь,время,алексий,нобелевский,война,пр..."


In [291]:
data['Topic'] = data['Topic'].fillna("")
#set(data['Topic'])
for i in tqdm(indexes):
    news_topics =  data['Topic'][i]
    data['Topic'][i] = news_topics.split("\n")
data.head(3)

  0%|                                                                                          | 0/218 [00:00<?, ?it/s]C:\Users\Nuts\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
100%|████████████████████████████████████████████████████████████████████████████████| 218/218 [00:43<00:00,  5.09it/s]


,Header,Content,Time,Link,Label,Topic,keywords
0,"В одном из ресторанов Нью-Йорка появится ""пяти...",Нью-йоркский ресторан Lucy's Cantina Royale бу...,6/10/2017,https://tass.ru/obschestvo/4625856,0,[Политика],"пауть,острый,президент,владимир,честь,ресторан..."
1,Экс-глава МИ-6 рассказал о провальном плане за...,Бывший глава британской разведки МИ-6 Джон Ска...,15/08/2017,http://ren.tv/novosti/2017-08-15/eks-glava-mi-...,0,[Политика],"россия,стратегический план,развал,план,стратег..."
2,Умерла нобелевский лауреат по литературе Светл...,Нобелевский лауреат по литературе Светлана Але...,18/05/2017,https://lenta.ru/news/2017/05/18/alex_nob/,0,[Знаменитости],"еврей,жизнь,время,алексий,нобелевский,война,пр..."


In [286]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
lem = WordNetLemmatizer()
stem = PorterStemmer()
word = "inversely"
stop_words = set(stopwords.words('russian'))
new_words = ['что', 'это', 'так', 'вот', 'ещё', 'свой', 'быть', 'й', 'как', 'в', 'б','наш', 'к', 'на', 'свой',
             'который', 'очень', 'которые', 'кстати', 'также', 'весь','риа','и']

stop_words = stop_words.union(new_words)
def processing(texts):
    #for i in tqdm_notebook(range(0, len(texts))):
        #Remove punctuations

        text = re.sub(r'\W', ' ', texts)
        #print(text)
        #Convert to lowercase
        text = text.lower()

        #remove tags
        text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)

        # remove special characters and digits
        text=re.sub("(\\d|\\W)+"," ",text)

        ##Convert to list from string
        text = text.split()

        ## pymorphy
        analyzer = pymorphy2.MorphAnalyzer()  
        corpus = (analyzer.normal_forms(word)[0] for word in text)
        
        lem = WordNetLemmatizer()
        corpus = [lem.lemmatize(word) for word in corpus if not word in  
                stop_words]
        return corpus



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nuts\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## topic 

In [292]:
#
for j in tqdm(indexes):
        topics = []
        t = data['Topic'][j]
        for i in t:
            i = i.replace(' Украина', 'Украина')
            i = i.replace(' Социальные сети', 'Социальные сети')
            i = i.replace('Социальные сети', 'Интернет')
            i = i.replace('УKраина', 'Украина')
            i = i.replace('УКраина', 'Украина')
            i = i.replace('Полтика', 'Политика')
            i = i.replace('Политка', 'Политика')
            i = i.replace('Сирия', 'Сирия')
            i = i.replace('Визы в США', 'США')
            i = i.replace('Путинбургер', 'Путин')
            topics.append(i)
        data['Topic'][j] = topics
        
        

  0%|                                                                                          | 0/218 [00:00<?, ?it/s]C:\Users\Nuts\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|████████████████████████████████████████████████████████████████████████████████| 218/218 [00:43<00:00,  5.01it/s]


In [293]:
topics = []
for t in data['Topic']:
    topics += t
topics = list(set(topics))
topics


['США',
 'Спорт',
 '',
 'Армия',
 'Экономика',
 'Скрипали',
 'Трагедия в кемерово',
 'Украина',
 'Церковь',
 'Политика ',
 'Санкции',
 'Сирия',
 'Выборы',
 'Сирия ',
 'Малазийский боинг',
 'Путин',
 'Великобритания',
 'Образование',
 'Крым',
 'Интернет',
 'Знаменитости',
 'ВЫборы',
 'Политика',
 'Общество',
 'Технологии',
 'Германия',
 'Россия']

In [294]:
# dict news _ topic
news_clast = defaultdict(list)
for i in indexes:
    t = data['Topic'][i]
    for word in t:
        news_clast[word].append(i)

In [262]:
DocEntry = namedtuple('DocEntry',['doc_id','positions'])
class InvertedIndex:
    
    def __init__(self):
        self.dict = defaultdict(list)
        self.texts = dict()
        
    def add_document(self, text, doc_id):
        self.texts[doc_id] = text
        words = processing(text)
        
        
        word_to_entry = defaultdict(lambda: [])
        
        for pos, word in enumerate(words):
            
            doc_entry = word_to_entry[word]
            doc_entry.append(pos)
            
        for word, positions in word_to_entry.items():
            postings = self.dict[word]
            entry = DocEntry(doc_id, positions)
            
            i = bisect.bisect_left(postings, entry)
        
            postings.insert(i, entry)
            
    def get_postings(self, word):
        return self.dict[word]


In [295]:
#w_d список документов со словом w
#d_cl список документов из класca t 

#w_d,  d_cl
def f_N11( w_d,d_cl,n):
    k= 0
    
    for d in range(n):
        if (d in d_cl )& (d in w_d):
            k+=1
    return k
#not w_d, w_cl
def f_N01(w_d,w_cl,n):
    k= 0
    for d in range(n):
        if (d in w_cl )& (d not in w_d):
            k+=1
    return k
#not w_d, not w_cl
def f_N00( w_d,w_cl,n):
    k= 0
    for d in range(n):
        if (d not in w_cl )& (d not in w_d):
            k+=1
    return k

#w_d,   not w_cl
def f_N10( w_d,w_cl,n):
    k= 0
    for d in range(n):
        if (d not in w_cl )& (d in w_d):
            k+=1
    return k

In [296]:

#analyzer = pymorphy2.MorphAnalyzer()   
"""Создание инвертированного индекса"""   
index = InvertedIndex()

for i in tqdm_notebook(indexes):
        doc_id = i
        text = data['Content'][i]
        index.add_document(text,doc_id)

# Mutual Information

In [265]:
def mutual_information(t):
    MI = []
    N = len(data['Content'])
    for word in (word_dict.items()):
        w  = word[1]
        
        w_d=[]
        for doc_id,pos in index.dict.get(w):
            w_d.append(doc_id)
        
        N11 = f_N11(w_d,news_clast.get(t),N)
        N1 = len(w_d)
        N10 = f_N10(w_d,news_clast.get(t),N)
        N0 = N - N1
        #print(N, N1, N0)
        N01 = f_N01(w_d,news_clast.get(t),N)
        N00 = f_N00(w_d,news_clast.get(t),N)
        if N10==0: N10=0.0001
        if N01==0: N01=0.0001
        if N11==0: N11=0.0001
        if N00==0: N00=0.0001
   
        #print(N11, N01, N10, N00)
        I = N11/N * math.log2(N*N11/(N1*N1)) + N01/N*math.log2(N * N01/(N0*N1)) +N10/N*math.log2(N * N10/(N0*N1))+N00/N*math.log2(N * N00/(N0*N0))
        MI.append(I)
    return MI

In [266]:
#словарь слов: ключ - номер индекса , значение -словао
word_dict = {}
for i, word in enumerate(index.dict.keys()):
    word_dict[i] = word

In [199]:
index.dict.get("порошенко")

[DocEntry(doc_id=20, positions=[27]),
 DocEntry(doc_id=21, positions=[57]),
 DocEntry(doc_id=23, positions=[73]),
 DocEntry(doc_id=32, positions=[99]),
 DocEntry(doc_id=38, positions=[1, 36, 70, 151]),
 DocEntry(doc_id=40, positions=[31, 47]),
 DocEntry(doc_id=72, positions=[6, 22, 41, 57, 65, 84, 137]),
 DocEntry(doc_id=79, positions=[19]),
 DocEntry(doc_id=88, positions=[47]),
 DocEntry(doc_id=134, positions=[8, 34, 49]),
 DocEntry(doc_id=172, positions=[79, 140]),
 DocEntry(doc_id=175, positions=[8]),
 DocEntry(doc_id=204, positions=[30, 76]),
 DocEntry(doc_id=209, positions=[4])]

# Mutual Information : 
### MI(word, topic) for all words, for all topics


In [267]:

data_mi = pd.DataFrame(index=range(len(word_dict.items())),columns = topics)
for topic in tqdm_notebook(topics):
    MI = mutual_information(topic)
    for i, mi in enumerate(MI):
        data_mi[topic][i] = mi

In [297]:
data_mi.head(3)

,США,Спорт,,Армия,Экономика,Скрипали,Трагедия в кемерово,Украина,Церковь,Политика,...,Образование,Крым,Интернет,Знаменитости,ВЫборы,Политика,Общество,Технологии,Германия,Россия
0,0.00794302,0.003376,0.0493669,0.03357,0.03357,0.0147494,0.00936716,0.266747,0.00201208,0.03357,...,0.0224124,0.0147494,0.00568916,0.0164324,0.03357,0.633258,0.152763,0.03357,0.0224124,0.00568916
1,0.0784185,0.0177657,0.192026,0.00210369,0.00210369,0.00162823,0.00541716,0.610537,0.0347377,0.00210369,...,0.000119106,0.00162823,0.0109095,0.00541716,0.00210369,1.22596,0.405202,0.00210369,0.000119106,0.0109095
2,0.0258607,0.0015669,0.0896516,0.0162167,0.0162167,0.00361519,0.00134177,0.376469,0.0064174,0.0162167,...,0.00816886,0.00361519,0.000772154,0.00134177,0.0162167,0.832439,0.23158,0.0162167,0.00816886,0.000772154


### "main" words for topics
Сирия

In [269]:
topic = 'Сирия'
top_ind = data_mi.sort_values([topic],ascending=False)[5:20].index
for i in top_ind:
        print(word_dict[i])

человек
пастор
пропагандистка
запустить
сорок
джебхата
отправиться
ранение
дакниша
омрана
нуср
ан
шок
встретиться
баронесса


In [270]:
topic = 'Общество'
top_ind = data_mi.sort_values([topic],ascending=False)[0:20].index
for i in top_ind:
        print(word_dict[i])
        

осло
смерть
ходорковский
скупать
африка
орхан
джемаля
расторгуев
радченко
африканский
безответственно
юстиция
плохо
николь
авантюра
революционер
календарь
разоблачительный
гнев
беллуб


In [271]:
topic = 'Путин'
top_ind = data_mi.sort_values([topic],ascending=False)[5:20].index
for i in top_ind:
        print(word_dict[i])

украина
мочь
заявить
время
слово
сша
глава
дело
считать
сказать
президент
власть
американский
москва
говорить



add mi_keywords to dataframe

In [298]:
mi_keywords = pd.DataFrame(index = ["words"])
mi_keywords

""
words


In [273]:
for topic in topics:
    mi_keywords[topic] = ""
    top = data_mi.sort_values([topic],ascending=False)[:20].index
    words = []
    for i in top:
        words.append(word_dict[i])
    mi_keywords[topic]["words"] = words
mi_keywords
    


,США,Спорт,,Армия,Экономика,Скрипали,Трагедия в кемерово,Украина,Церковь,Политика,...,Образование,Крым,Интернет,Знаменитости,ВЫборы,Политика,Общество,Технологии,Германия,Россия
words,"[россия, год, российский, страна, украина, сша...","[россия, год, российский, страна, украина, чел...","[россия, год, мастер, письменный, углубление, ...","[россия, год, российский, страна, человек, укр...","[россия, год, российский, страна, человек, укр...","[россия, год, российский, страна, человек, укр...","[россия, год, российский, страна, человек, укр...","[предельный, отопительный, жилой, просеиваться...","[россия, год, российский, страна, украина, чел...","[россия, год, российский, страна, человек, укр...",...,"[россия, год, российский, страна, человек, укр...","[россия, год, российский, страна, человек, укр...","[россия, год, российский, страна, украина, чел...","[россия, год, российский, страна, человек, укр...","[россия, год, российский, страна, человек, укр...","[седвилл, медиаресурс, самоубийственный, озагл...","[осло, смерть, ходорковский, скупать, африка, ...","[россия, год, российский, страна, человек, укр...","[россия, год, российский, страна, человек, укр...","[россия, год, российский, страна, украина, чел..."


In [274]:
data["mi_keywords"]=""

In [278]:

for i in (range(len(data))):
    keywords = []
    topics = data["Topic"][i]
    for top in topics:
        #print(top, mi_keywords[top]["words"])
        keywords+=mi_keywords[top]["words"]
    data["mi_keywords"][i] = keywords

C:\Users\Nuts\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [281]:
data.head(3)

,Header,Content,Time,Link,Label,Topic,keywords,mi_keywords
0,"В одном из ресторанов Нью-Йорка появится ""пяти...",Нью-йоркский ресторан Lucy's Cantina Royale бу...,6/10/2017,https://tass.ru/obschestvo/4625856,0,[Политика],"пауть,острый,президент,владимир,честь,ресторан...","[седвилл, медиаресурс, самоубийственный, озагл..."
1,Экс-глава МИ-6 рассказал о провальном плане за...,Бывший глава британской разведки МИ-6 Джон Ска...,15/08/2017,http://ren.tv/novosti/2017-08-15/eks-glava-mi-...,0,[Политика],"россия,стратегический план,развал,план,стратег...","[седвилл, медиаресурс, самоубийственный, озагл..."
2,Умерла нобелевский лауреат по литературе Светл...,Нобелевский лауреат по литературе Светлана Але...,18/05/2017,https://lenta.ru/news/2017/05/18/alex_nob/,0,[Знаменитости],"еврей,жизнь,время,алексий,нобелевский,война,пр...","[россия, год, российский, страна, человек, укр..."


In [282]:
data.to_csv('fake news keywords.csv', sep='\t', encoding='utf-8')